## Extract data from pv and nwp

Extract big data from pv and nwp, by randomly choosing dates, and randomly choosing latitude and longitude. 

For those random points, choose, all steps and make a dataset. 

- select date, of day and then can try data.load to load data locally
- function in xarray, toDataframe(), can get dataframes, from xarray directly 
- and then concat for all days in a month. 

- randomly take some location and some times
- size of data: 1000 init times * 49 steps 
- 20, to 50 different sites. 

- can get 10 to 15 columns of ss_id and weather variables
- just keep ss_id, in resulting dataframe
- can get lat, long, capacity from metadata when need to feed to the model
- so our resulting dataframe will have ss_id and weather variables 10 to 15 columns


In [6]:
# select generation from pv_data , for each variable ss_id, and each date

# init time and daetime are not exact same from pv to nwp

# need to check nearest for datetime, what other options are available, 

# init time should be less than datetime,
# 
# 
# 
# training and testing, can keep 10 ids for trainign, 5 for inference
# 
# make your own repo on github and upload the code, for zak
# 
# use pv 30 min parquet file, and then select randomly . it is 9 gb
# pv.resample, data make it half an hour. get teh avergae of 6 values from 5 minutes each and use that vaue for 30 mins geenration. 
#  try to resample current one. 
#  

## Problems to fix
- nwp data has init_time, which is only a date and not hours, steps have hour for them, so nwp data is every hour. the init_time has either T00 or T12. Every date has T00 and T12, the program treats both dates separately. 

- pv has data for every date, every 5 minutes geenration captured. 

- so right now, when we are using nearest for datetime and init_time, it gets, one datetime from pv and keep it same for one date of init_time. 

- now when we selecting datetime from pv and then init_time from nwp, select first from pv, then find the time less than datetime from nwp. from nwp, can be less than, but can't be more than nwp time. 

- now problemis how to get time from nwp, need to consider both init_time and steps. 

- let's take for an hour from pv. 

## confusion
we are only selecting date from pv data sets, because for that date we are taking all steps, at every hour from nwp. if we select hour too, we need to set step too at that fixed time. so it mean, when i select a date from pv data, i need to get all hours data from it. 

In [10]:
import xarray as xr
# import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import plotly.express as px
import numpy as np


In [11]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [12]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

In [13]:

meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


#### first step

- need to do one thing, need to take every half an hour data
- because in nwp data is every half an hour

- randomly choose pv sites, did it manually
- also need to select date time from pv_site
- randomly select 3 values from pv_site for each ss_id
- ss_id can be a key of dictionary, and date, latitude, longitude can be values
- date is not going to be one value liek latitude and long, it is going to be mulitple values 
- so i can maintain two dictionaries too, one with lat, long, and one with date time values
- i should make one dataframe with lat , long , is and dates. 
- and then go to nwp and get the values of variables in it.

- need to get 3 things from pv_data, ss_id, datetime and generation. generation is amount of power generated at that time. 

In [29]:
pv_data
# pv_sites_id = [10064, 10168, 10528, 12091, 17073, 26846, 2812, 7050, 9902, 3324]
# print(pv_sites_id)

hourly_pv_data = pv_data.sel(datetime = pv_data['datetime'].dt.minute == 0)

data_vars = list(pv_data.data_vars)
pv_sites_id = np.random.choice(data_vars, 25, replace=False)
print(pv_sites_id)

data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : []}

for ss_id in pv_sites_id:
    random_dates = np.random.choice(hourly_pv_data['datetime'].values, 700, replace=False)
    power_values = hourly_pv_data[ss_id].sel(datetime=random_dates).values

    for date, power in zip(random_dates, power_values):
        data_dict['ss_id'].append(int(ss_id))
        data_dict['pv_datetime'].append(date)
        data_dict['generation'].append(power)




    # data_dict['ss_id'].extend([int(ss_id)] * len(random_dates))
    # data_dict['date'].extend(random_dates)
    # data_dict['generation'].extend(power_values)
    
   

    # taking too much time, this apporach and loop
    # for date in random_dates:
    #     data_dict['ss_id'].append(int(ss_id))
    #     data_dict['date'].append(date)

    #     power = pv_data.sel(datetime=date)[ss_id].values
    #     data_dict['generation'].append(power)

pv_df = pd.DataFrame(data_dict)

pv_df = pv_df.dropna(subset={'generation'})

pv_df


['6498' '7017' '27042' '3242' '3536' '10586' '26844' '9569' '11465' '7295'
 '27065' '6602' '8215' '6573' '11466' '2903' '7551' '5358' '26877' '6630'
 '13768' '12035' '7885' '26842' '6681']


,ss_id,pv_datetime,generation
0,6498,2019-03-01 03:00:00,0.000000
1,6498,2018-11-04 18:00:00,0.000000
2,6498,2019-11-27 00:00:00,0.000000
3,6498,2018-07-13 16:00:00,792.690002
4,6498,2020-07-12 04:00:00,0.000000
...,...,...,...
17481,6681,2018-06-14 12:00:00,1338.180054
17493,6681,2018-05-11 05:00:00,81.047760
17495,6681,2018-04-20 12:00:00,2356.031982
17496,6681,2018-08-23 10:00:00,384.420013


In [30]:
pv_df.to_csv('result_data/pv_generaton.csv')


#### second step to find random dates and latitude, longitude for these sites. 
- that can be done from metadata
- one approach is to make a new dictioanry with key as ss_id an dpngitude and latitiude as values elements. 
- or make a dictionary and then make a dataframe out of it.


In [31]:
pv_sites_id = [int(id) for id in pv_sites_id]  
# pv_sites_id

In [32]:
pv_site_dict = {'ss_id':[], "lat":[], "long": []}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
    else:
        print('row empty')
        


print(pv_site_dict)
meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
meta_site_df

{'ss_id': [6498, 7017, 27042, 3242, 3536, 10586, 26844, 9569, 11465, 7295, 27065, 6602, 8215, 6573, 11466, 2903, 7551, 5358, 26877, 6630, 13768, 12035, 7885, 26842, 6681], 'lat': [50.78, 53.98, 51.91, 56.43, 51.95, 56.97, 53.5, 56.23, 51.77, 50.34, 51.36, 57.67, 51.16, 55.96, 57.66, 51.94, 50.83, 52.58, 52.5, 51.31, 53.55, 55.97, 54.94, 53.45, 53.03], 'long': [-1.96, -1.11, -0.67, -3.4, -0.29, -2.23, -2.35, -4.2, -3.1, -4.79, -2.93, -2.33, -3.32, -2.97, -3.01, -0.88, -2.31, -0.29, -1.79, -0.57, -2.79, -3.7, -2.92, -2.69, -1.21]}


,ss_id,lat,long
0,6498,50.78,-1.96
1,7017,53.98,-1.11
2,27042,51.91,-0.67
3,3242,56.43,-3.40
4,3536,51.95,-0.29
5,10586,56.97,-2.23
6,26844,53.50,-2.35
7,9569,56.23,-4.20
8,11465,51.77,-3.10
9,7295,50.34,-4.79


In [33]:
# combine both dates, with lat and long of pv site

combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df


,ss_id,pv_datetime,generation,lat,long
0,6498,2019-03-01 03:00:00,0.000000,50.78,-1.96
1,6498,2018-11-04 18:00:00,0.000000,50.78,-1.96
2,6498,2019-11-27 00:00:00,0.000000,50.78,-1.96
3,6498,2018-07-13 16:00:00,792.690002,50.78,-1.96
4,6498,2020-07-12 04:00:00,0.000000,50.78,-1.96
...,...,...,...,...,...
12119,6681,2018-06-14 12:00:00,1338.180054,53.03,-1.21
12120,6681,2018-05-11 05:00:00,81.047760,53.03,-1.21
12121,6681,2018-04-20 12:00:00,2356.031982,53.03,-1.21
12122,6681,2018-08-23 10:00:00,384.420013,53.03,-1.21


#### Third step
- now we have a dictionay of random pv_sites and their latitude and longitude.
- next step is to go to nwp, sleect 10 variables and for those 10 variables select values
- go through the dictionary and select all steps and 10 variables. 

In [35]:
print(combined_df.shape)
# ['t2m', 'dswrf', 'dlwrf', 'sde', 'sr', 'tcc', 'hcc', 'lcc', 'mcc',
#    'duvrs', 'u10', 'v10', 'u100', 'v100']

# List to collect DataFrames
dataframes = []

(12124, 5)


- 10 sites, 3 datetimes, 30 rows
- 30* 49 steps = 1470

In [38]:
for index, row in combined_df.iterrows():
    ss_id = row['ss_id']
    data_sel = nwp_data.sel(latitude=row['lat'], method="nearest") \
                       .sel(longitude=row['long'], method="nearest") \
                       .sel(init_time=row['pv_datetime'], method="nearest") # need to change that nethod = "backfill"

    data_df = data_sel.to_dataframe().reset_index()
    print(data_df)
    print("printed data df")
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

    pivot_df['ss_id'] = ss_id
    pivot_df['datetime'] = row['pv_datetime']
    print(pivot_df)

    dataframes.append(pivot_df)
    print(pivot_df.head())
    break

    variable            step    ECMWF_UK  init_time  latitude  longitude
0        t2m 0 days 00:00:00  282.065918 2019-03-01      50.8      -1.95
1        t2m 0 days 01:00:00  281.462158 2019-03-01      50.8      -1.95
2        t2m 0 days 02:00:00  281.336426 2019-03-01      50.8      -1.95
3        t2m 0 days 03:00:00  281.115234 2019-03-01      50.8      -1.95
4        t2m 0 days 04:00:00  280.350342 2019-03-01      50.8      -1.95
..       ...             ...         ...        ...       ...        ...
681     v100 1 days 20:00:00    9.542358 2019-03-01      50.8      -1.95
682     v100 1 days 21:00:00    9.079600 2019-03-01      50.8      -1.95
683     v100 1 days 22:00:00    7.999947 2019-03-01      50.8      -1.95
684     v100 1 days 23:00:00    6.679502 2019-03-01      50.8      -1.95
685     v100 2 days 00:00:00    4.476947 2019-03-01      50.8      -1.95

[686 rows x 6 columns]
printed data df
variable  init_time            step         dlwrf         dswrf         duvrs  \
0  

In [ ]:
final_df = pd.concat(dataframes, ignore_index=True)
print(final_df.shape)

In [116]:

desired_order = ['ss_id', 'init_time', 'step', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100', 'datetime']
final_df = final_df[desired_order]


- 25 pv sites
- 500 dates for each
- 25 * 500 = 12500 rows
- 12500 * 49 steps = 612500 rows
- took 15 minutes for 612500 25 sites, 500 dates each
- each site has 24500 rows, 1* 500 * 49 = 24,500

In [115]:
final_df.head()

variable,init_time,step,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,ss_id,datetime
0,2019-09-22 12:00:00,0 days 00:00:00,0.0,0.000000e+00,0.00000,0.982452,0.013000,0.933258,0.0,0.0,291.135742,0.992920,-2.428142,-3.754718,-1.049166,0.154392,7547,2019-09-22 09:15:00
1,2019-09-22 12:00:00,0 days 01:00:00,1353008.0,5.504762e+05,71664.53125,0.933594,0.001801,0.986847,0.0,250496.0,290.958496,0.998718,-3.026777,-4.564105,-0.490215,1.016706,7547,2019-09-22 09:15:00
2,2019-09-22 12:00:00,0 days 02:00:00,2712773.0,1.003392e+06,141429.59375,0.362244,0.000092,0.777679,0.0,334080.0,290.792236,0.841644,-3.483191,-5.351845,-0.659222,0.992348,7547,2019-09-22 09:15:00
3,2019-09-22 12:00:00,0 days 03:00:00,4054373.0,1.545526e+06,218643.50000,0.110321,0.009705,0.680908,0.0,677632.0,291.014404,0.703888,-3.351099,-5.051410,0.102624,2.067908,7547,2019-09-22 09:15:00
4,2019-09-22 12:00:00,0 days 04:00:00,5361400.0,2.148786e+06,286584.31250,0.503754,0.010254,0.870911,0.0,1538816.0,291.002930,0.923370,-2.785363,-3.987979,0.761361,3.078596,7547,2019-09-22 09:15:00


In [117]:
row = final_df[final_df['ss_id'] == 7547]
print(row.shape)
row.head()

(24500, 18)


variable,ss_id,init_time,step,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,datetime
0,7547,2019-09-22 12:00:00,0 days 00:00:00,0.0,0.000000e+00,0.00000,0.982452,0.013000,0.933258,0.0,0.0,291.135742,0.992920,-2.428142,-3.754718,-1.049166,0.154392,2019-09-22 09:15:00
1,7547,2019-09-22 12:00:00,0 days 01:00:00,1353008.0,5.504762e+05,71664.53125,0.933594,0.001801,0.986847,0.0,250496.0,290.958496,0.998718,-3.026777,-4.564105,-0.490215,1.016706,2019-09-22 09:15:00
2,7547,2019-09-22 12:00:00,0 days 02:00:00,2712773.0,1.003392e+06,141429.59375,0.362244,0.000092,0.777679,0.0,334080.0,290.792236,0.841644,-3.483191,-5.351845,-0.659222,0.992348,2019-09-22 09:15:00
3,7547,2019-09-22 12:00:00,0 days 03:00:00,4054373.0,1.545526e+06,218643.50000,0.110321,0.009705,0.680908,0.0,677632.0,291.014404,0.703888,-3.351099,-5.051410,0.102624,2.067908,2019-09-22 09:15:00
4,7547,2019-09-22 12:00:00,0 days 04:00:00,5361400.0,2.148786e+06,286584.31250,0.503754,0.010254,0.870911,0.0,1538816.0,291.002930,0.923370,-2.785363,-3.987979,0.761361,3.078596,2019-09-22 09:15:00


In [118]:
final_df.shape

(612500, 18)

In [119]:
final_df.to_csv("result_data/final_data.csv")

In [121]:
final_df.head()

variable,ss_id,init_time,step,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,datetime
0,7547,2019-09-22 12:00:00,0 days 00:00:00,0.0,0.000000e+00,0.00000,0.982452,0.013000,0.933258,0.0,0.0,291.135742,0.992920,-2.428142,-3.754718,-1.049166,0.154392,2019-09-22 09:15:00
1,7547,2019-09-22 12:00:00,0 days 01:00:00,1353008.0,5.504762e+05,71664.53125,0.933594,0.001801,0.986847,0.0,250496.0,290.958496,0.998718,-3.026777,-4.564105,-0.490215,1.016706,2019-09-22 09:15:00
2,7547,2019-09-22 12:00:00,0 days 02:00:00,2712773.0,1.003392e+06,141429.59375,0.362244,0.000092,0.777679,0.0,334080.0,290.792236,0.841644,-3.483191,-5.351845,-0.659222,0.992348,2019-09-22 09:15:00
3,7547,2019-09-22 12:00:00,0 days 03:00:00,4054373.0,1.545526e+06,218643.50000,0.110321,0.009705,0.680908,0.0,677632.0,291.014404,0.703888,-3.351099,-5.051410,0.102624,2.067908,2019-09-22 09:15:00
4,7547,2019-09-22 12:00:00,0 days 04:00:00,5361400.0,2.148786e+06,286584.31250,0.503754,0.010254,0.870911,0.0,1538816.0,291.002930,0.923370,-2.785363,-3.987979,0.761361,3.078596,2019-09-22 09:15:00


In [3]:
df = pd.read_csv("result_data/final_data.csv")

In [4]:
df

,Unnamed: 0,ss_id,init_time,step,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,datetime
0,0,7547,2019-09-22 12:00:00,0 days 00:00:00,0.0,0.00,0.00,0.982452,0.013000,0.933258,0.0,0.0,291.13574,0.992920,-2.428142,-3.754718,-1.049166,0.154392,2019-09-22 09:15:00
1,1,7547,2019-09-22 12:00:00,0 days 01:00:00,1353008.0,550476.20,71664.53,0.933594,0.001801,0.986847,0.0,250496.0,290.95850,0.998718,-3.026777,-4.564105,-0.490215,1.016706,2019-09-22 09:15:00
2,2,7547,2019-09-22 12:00:00,0 days 02:00:00,2712773.0,1003391.56,141429.60,0.362244,0.000092,0.777679,0.0,334080.0,290.79224,0.841644,-3.483191,-5.351845,-0.659222,0.992348,2019-09-22 09:15:00
3,3,7547,2019-09-22 12:00:00,0 days 03:00:00,4054373.0,1545525.80,218643.50,0.110321,0.009705,0.680908,0.0,677632.0,291.01440,0.703888,-3.351099,-5.051410,0.102624,2.067908,2019-09-22 09:15:00
4,4,7547,2019-09-22 12:00:00,0 days 04:00:00,5361400.0,2148786.50,286584.30,0.503754,0.010254,0.870911,0.0,1538816.0,291.00293,0.923370,-2.785363,-3.987979,0.761361,3.078596,2019-09-22 09:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612495,612495,7674,2021-08-18 12:00:00,1 days 20:00:00,54739424.0,15279508.00,1944450.00,0.296600,0.996166,0.813415,0.0,9847630.0,287.80078,0.997953,-2.427940,-3.772998,0.109574,1.472033,2021-08-18 09:05:00
612496,612496,7674,2021-08-18 12:00:00,1 days 21:00:00,56087090.0,15865853.00,2005971.00,0.406067,0.980194,0.328979,0.0,9989082.0,288.86646,0.986301,-2.700450,-3.596149,0.657565,1.309469,2021-08-18 09:05:00
612497,612497,7674,2021-08-18 12:00:00,1 days 22:00:00,57438030.0,16558208.00,2130966.00,0.201508,0.939758,0.703522,0.0,10195067.0,289.50098,0.960547,-2.965534,-3.919898,1.002465,1.714531,2021-08-18 09:05:00
612498,612498,7674,2021-08-18 12:00:00,1 days 23:00:00,58784050.0,17839480.00,2257726.00,0.890472,0.998108,0.966186,0.0,10777962.0,290.83740,0.999978,-2.145883,-2.850034,2.947081,3.983860,2021-08-18 09:05:00
